Program 1

Christian Basso, Patrick McDonald, David Litton

CSC 3310

January 30, 2024

# Program 1

This code will take in a list of tuples that represent a point with an x and y coordinate. It will create the largest convex hull possible from the points and return the list of all the points that represent the hull.

Here are some array examples:

In [1]:
arr1 = ((0,0), (1, 1), (2, 2), (3, 3), (1, 4), (4, 5), 
        (5, 6), (6, 7), (7, 8), (8, 9), (0, 5), (9, 0), 
        (4, 9), (0, 6), (6, 9), (0, 7), (7, 9), (0, 8), 
        (8, 9), (0, 9), (7, 9), (6, 2))

arr2 = ((15, 5), (8, 29), (4, 22), (26, 19), (10, 5), 
        (21, 7), (19, 7), (30, 25), (14, 28), (19, 12))

arr3 = ((8, 16), (22, 7), (23, 27), (14, 16), (9, 20), 
        (29, 30), (10, 17), (20, 21), (4, 12), (29, 5))

arr4 = ((0, 0), (1, 1), (2, 2), (3, 3)) #This one shouldnt work...

arr5 = ((0, 0), (1, 1)) #This one shouldnt work...

In [ ]:
#(David)
# def hull_algorithim (list of points)
# Find the furthest point
    # Calculate center average of all points
    # Take point with longest line from the center
# Set furthest point as current point

#(Christian)
# define function to find most obtuse angle between current point and all other points
    # for each point
        # cacluate angle between current point and all other points
        # Return most obtuse point

# (Patrick)
# for each point
    # Find most obtuse point
    # Add current point to list of hull points
    # Make most obtuse angle point the current point
# return hull points